<a href="https://colab.research.google.com/github/Megaprathapan/machine-Learning/blob/RandomForest/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
import pandas as pd

In [157]:
df = pd.read_csv('https://raw.githubusercontent.com/santhoshkumaroff/MachineLearningDatasets/refs/heads/main/dataset_traffic_accident_prediction1.csv')

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Weather               798 non-null    object 
 1   Road_Type             798 non-null    object 
 2   Time_of_Day           798 non-null    object 
 3   Traffic_Density       798 non-null    float64
 4   Speed_Limit           798 non-null    float64
 5   Number_of_Vehicles    798 non-null    float64
 6   Driver_Alcohol        798 non-null    float64
 7   Accident_Severity     798 non-null    object 
 8   Road_Condition        798 non-null    object 
 9   Vehicle_Type          798 non-null    object 
 10  Driver_Age            798 non-null    float64
 11  Driver_Experience     798 non-null    float64
 12  Road_Light_Condition  798 non-null    object 
 13  Accident              798 non-null    float64
dtypes: float64(7), object(7)
memory usage: 92.0+ KB


In [159]:
df.head()

,Weather,Road_Type,Time_of_Day,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,Driver_Experience,Road_Light_Condition,Accident
0,Rainy,City Road,Morning,1.0,100.0,5.0,0.0,NaN,Wet,Car,51.0,48.0,Artificial Light,0.0
1,Clear,Rural Road,Night,NaN,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,43.0,Artificial Light,0.0
2,Rainy,Highway,Evening,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,52.0,Artificial Light,0.0
3,Clear,City Road,Afternoon,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,31.0,Daylight,0.0
4,Rainy,Highway,Morning,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,55.0,Artificial Light,1.0


In [160]:
from sklearn.model_selection import train_test_split

In [161]:
# df['Accident']=df['Accident'].fillna(df['Accident'].mean())
df.dropna(subset=['Accident'],inplace=True)


In [162]:
x=df.drop('Accident',axis=1)
y=df['Accident']
xtarin,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.8,random_state=42)


In [163]:
cat_col=x.select_dtypes(include='object').columns
cat_col

Index(['Weather', 'Road_Type', 'Time_of_Day', 'Accident_Severity',
       'Road_Condition', 'Vehicle_Type', 'Road_Light_Condition'],
      dtype='object')

In [164]:
num_col=x.select_dtypes(include='float64').columns
num_col

Index(['Traffic_Density', 'Speed_Limit', 'Number_of_Vehicles',
       'Driver_Alcohol', 'Driver_Age', 'Driver_Experience'],
      dtype='object')

In [165]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [166]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer

In [167]:


num_pipeline=Pipeline(steps=[
    ('num_col',SimpleImputer(strategy='mean')),
        ('scaler',StandardScaler())
])

cat_pipeline=Pipeline(steps=[
    ('cat_col',SimpleImputer (strategy='most_frequent')),
    ('onehot',OneHotEncoder())
])

In [168]:
preprocessing=ColumnTransformer(
    transformers=[
         ('num',num_pipeline,num_col),
         ('cat',cat_pipeline,cat_col)
           ])

In [169]:
from sklearn.ensemble import RandomForestClassifier

In [170]:
Pipeline=Pipeline(steps=[
    ('preprocessing',preprocessing),
    ('model',RandomForestClassifier())
])

In [171]:
from sklearn.model_selection import GridSearchCV

In [172]:
params = {
    'model__criterion' :['entropy','gini'],
    'model__max_depth' :[5,10,50,100,200],
    'model__min_samples_leaf' :[2,3,5,7,10],
    'model__min_samples_split' :[2,3,5,10]
  }

gridsearch = GridSearchCV (Pipeline,params,cv=5,n_jobs=-1)

In [173]:
gridsearch.fit(xtarin,ytrain)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('num_col',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['Traffic_Density', 'Speed_Limit', 'Number_of_Vehicles',
       'Driver_Alcohol', 'Driver_Age', 'Driver_Experience'],
      dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('cat_col',
                                                                                          SimpleImputer(...
                                                                         Index(['Weather', 'Road_Type', 'Time_of_Day', 'Accident_Severity',
       'Road_Condition', 'Vehicle_Type', 'Road_Light_Condition'],
      dtype='object'))])),
                                       ('model', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'model__criterion': ['entropy', 'gini'],
                         'model__max_depth': [5, 10, 50, 100, 200],
                         'model__min_samples_leaf': [2, 3, 5, 7, 10],
                         'model__min_samples_split': [2, 3, 5, 10]})

In [175]:
gridsearch.best_params_

{'model__criterion': 'entropy',
 'model__max_depth': 100,
 'model__min_samples_leaf': 3,
 'model__min_samples_split': 3}

In [176]:
gridsearch.best_score_

np.float64(0.7053518700787402)

In [177]:
gridsearch.score(xtarin,ytrain)

0.8103448275862069

In [178]:
gridsearch.score(xtest,ytest)

0.70625